<a href="https://colab.research.google.com/github/Annesya/voice-speech-metamers/blob/dev_branch/embedding_extraction_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor


In [2]:
!pip install speechbrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 11.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [3]:
from collections import defaultdict

import torch
from transformers import Wav2Vec2Model
from speechbrain.inference.encoders import WaveformEncoder


class Encoder(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.model_name = model_name
        if self.model_name == "wav2vec2_orig":
            self.model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        else:
            self.model = WaveformEncoder.from_hparams(source=ckpt, savedir=cache_dir, run_opts={"device":"cuda"})
            self.activation = defaultdict(list)
            for layer in range(12):
                getattr(self.model.mods.encoder.encoder_wrapper.latent_encoder.layers, str(layer)).register_forward_hook(self.get_layer_embeddings(f'encoder_features_{layer}'))

    def get_layer_embeddings(self, name):
        def hook(model, input, output):
            self.activation[name].append(output[0])
        return hook

    def forward(self, input_values):
        device = input_values[0].device
        if self.model_name == "wav2vec2_orig":
            output_values = [self.model(input_.squeeze(0), output_hidden_states=True) for input_ in input_values]
            return [list(output_.hidden_states[1:]) for output_ in output_values]
        else:
            inputs_len = torch.FloatTensor([len(input_) for input_ in input_values]).to(device)
            inputs_len_norm = inputs_len/inputs_len.max()
            _ = [self.model.encode_batch(input_.squeeze(0), inputs_len_norm[i])["embeddings"].squeeze(0) for i, input_ in enumerate(input_values)]
            # return self.activation
            output_values = []
            for i in range(len(input_values)):
                output_values.append([self.activation[name][i] for name in self.activation.keys()])
            self.activation = defaultdict(list)
            return output_values


if __name__ == '__main__':
    model = Encoder("wav2vec2_orig")
    # model = Encoder("wav2vec2_orig", "facebook/wav2vec2-base", "/om2/user/gelbanna/model_cache")
    # x = {}
    x = [torch.ones((1,1,35000)), torch.ones((1,1,30000))]
    print(x)
    # compute the forward pass
    y = model(x)
    # print(y.keys(), len(y["encoder_features_0"]), y["encoder_features_0"][1].shape)
    print(len(y))
    print(len(y[0]), len(y[1]))
    print(y[0][0].shape, y[1][0].shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[tensor([[[1., 1., 1.,  ..., 1., 1., 1.]]]), tensor([[[1., 1., 1.,  ..., 1., 1., 1.]]])]
2
12 12
torch.Size([1, 109, 768]) torch.Size([1, 93, 768])
